# Group 5 - Module 6: Game Playing Systems

***
### Group Members:
* **Nils Dunlop, 20010127-2359, Applied Data Science, e-mail: gusdunlni@student.gu.se (16 hours)**
* **Francisco Erazo, 19930613-9214, Applied Data Science, e-mail: guserafr@student.gu.se (16 hours)**

#### **We hereby declare that we have both actively participated in solving every exercise. All solutions are entirely our own work, without having taken part of other solutions." (This is independent and additional to any declaration that you may encounter in the electronic submission system.)**

# Assignment 6
***

## Problem 1: Reading and Reflection
***
AlphaGo is a computer program designed to play the board game Go. It was created by DeepMind Technologies, now a part of Google. Go is known for its complex strategies and the vast number of possible moves, presenting a significant challenge for traditional AI methods. AlphaGo uses advanced deep neural networks and Monte Carlo Tree Search (MCTS), enabling it to learn from a large amount of data, including recorded games between human experts and games it played against itself. Through this learning process, AlphaGo developed judgment and intuition similar to human players, allowing it to accurately predict moves and game outcomes.

The architecture of AlphaGo includes policy networks that suggest probable next moves, and value networks that predict the game's winner from any position, marking a step forward in using AI to tackle complex problems. Its training involved supervised learning from games played by human experts and reinforcement learning from self-play. This approach enabled the system to continuously refine its strategies and adjust to new challenges. The integration of MCTS with neural networks enabled efficient exploration and evaluation of possible moves, balancing between relying on known effective strategies and exploring new ones.

AlphaGo's success was demonstrated by its 99.8% win rate against other Go programs and its historic victory over European Go champion Fan Hui 5-0, marking the first time a computer program defeated a professional human player in Go. Given Go's complexity compared to chess, this achievement was seen as a significant milestone in AI.

The strategies employed by AlphaGo, and its implications, extend beyond just games. They showcase the potential of deep learning and reinforcement learning to address complex issues in various fields.



## Problem 2: Implementation
***

## References
***


## Self Check
***
- Have you answered all questions to the best of your ability?
Yes, we have.
- Is all the required information on the front page, is the file name correct etc.?
Indeed, all the required information on the front page has been included.
- Anything else you can easily check? (details, terminology, arguments, clearly stated answers etc.?)
We have checked, and everything looks good.